<a href="https://colab.research.google.com/github/luisariass/inteligent-Agent-FOL/blob/main/db_Agent_fol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pyDatalog import pyDatalog
import sqlite3

In [ ]:
# Conexión a la base de datos
conn = sqlite3.connect('inventario.db')
cursor = conn.cursor()

In [ ]:
cursor.execute('''
    CREATE TABLE IF NOT EXISTS productos (
        nombre TEXT PRIMARY KEY,
        stock INTEGER,
        minimo INTEGER,
        costo FLOAT,
        ventas_mensuales INTEGER
    )
''')

In [ ]:
# Insertar datos de ejemplo (solo si la tabla está vacía)
cursor.execute("SELECT COUNT(*) FROM productos")
if cursor.fetchone()[0] == 0:
    cursor.executemany('''
        INSERT INTO productos (nombre, stock, minimo, costo, ventas_mensuales)
        VALUES (?, ?, ?, ?, ?)
    ''', [
        ('Martillo', 20, 30, 5.0, 120),      # Martillo: 120 ventas/mes
        ('Cemento', 49, 50, 15.0, 80),       # Cemento: 80 ventas/mes
        ('Tornillos', 500, 200, 2.5, 150),   # Tornillos: 150 ventas/mes
        ('Clavos', 110, 120, 5.0, 200),      # Clavos: 200 ventas/mes
        ('Pintura', 15, 20, 25.0, 30),       # Pintura: 30 ventas/mes
        ('Brochas', 30, 50, 8.0, 40),        # Brochas: 40 ventas/mes
        ('Alicates', 1, 15, 12.0, 20),      # Alicates: 20 ventas/mes
        ('Destornilladores', 25, 30, 7.0, 60), # Destornilladores: 60 ventas/mes
        ('Sierra', 5, 10, 20.0, 10),         # Sierra: 10 ventas/mes
        ('Lijas', 200, 300, 1.0, 250)        # Lijas: 250 ventas/mes
    ])
    conn.commit()

In [ ]:
# Configuración de pyDatalog
pyDatalog.clear()
pyDatalog.create_terms('''
    producto, stock_actual, minimo_stock, costo_pedido,
    necesita_reposicion, realizar_pedido, producto_critico,
    alta_rotacion, bajo_stock_alta_demanda, ventas_mensuales,
    X, Y, Z, Cantidad, V
''')

# Cargar datos desde SQLite a pyDatalog
cursor.execute("SELECT nombre, stock, minimo, costo, ventas_mensuales FROM productos")
for nombre, stock, minimo, costo, ventas in cursor.fetchall():
    +producto(nombre)
    +stock_actual(nombre, stock)
    +minimo_stock(nombre, minimo)
    +costo_pedido(nombre, costo)
    +ventas_mensuales(nombre, ventas)  # Cargar ventas mensuales

In [ ]:
# Reglas de inferencia
necesita_reposicion(X) <= (
    producto(X) &
    stock_actual(X, Y) &
    minimo_stock(X, Z) &
    (Y < Z)
)

realizar_pedido(X, Cantidad) <= (
    necesita_reposicion(X) &
    stock_actual(X, Y) &
    minimo_stock(X, Z) &
    (Cantidad == Z - Y)
)

producto_critico(X) <= (
    producto(X) &
    stock_actual(X, Y) &
    minimo_stock(X, Z) &
    (Y < 0.2 * Z)
)

alta_rotacion(X) <= (
    producto(X) &
    ventas_mensuales(X, V) &
    (V > 100)
)

bajo_stock_alta_demanda(X) <= (
    necesita_reposicion(X) &
    alta_rotacion(X)
)


bajo_stock_alta_demanda(X) <= necesita_reposicion(

In [ ]:
# Consultas
print("=== Productos a reponer ===")
print(necesita_reposicion(X))

print("\n=== Pedidos necesarios ===")
print(realizar_pedido(X, Cantidad))

print("\n=== Productos críticos ===")
print(producto_critico(X))

print("\n=== Productos con alta rotación ===")
print(alta_rotacion(X))

print("\n=== Productos con bajo stock y alta demanda ===")
print(bajo_stock_alta_demanda(X))

conn.close()

=== Productos a reponer ===
X               
----------------
Lijas           
Sierra          
Destornilladores
Alicates        
Brochas         
Pintura         
Clavos          
Cemento         
Martillo        

=== Pedidos necesarios ===
X                | Cantidad
-----------------|---------
Lijas            | 100     
Sierra           | 5       
Destornilladores | 5       
Alicates         | 14      
Brochas          | 20      
Pintura          | 5       
Clavos           | 10      
Cemento          | 1       
Martillo         | 10      

=== Productos críticos ===
X       
--------
Alicates

=== Productos con alta rotación ===
X        
---------
Lijas    
Clavos   
Tornillos
Martillo 

=== Productos con bajo stock y alta demanda ===
X       
--------
Lijas   
Clavos  
Martillo
